In [1]:
# 계층 단위로 구현하자
# 계층이란 -> 기능의 최소 단위로 덧셈, 곱셈, 활성화 함수, 행렬 곱 의미

class MulLayer: # 곱셈 노드
    def __init__(self): # 초기화 함수 => forward Pass인 경우, 입력값을 유지하기 위해서 = Input 저장 변수
        self.x = None
        self.y = None
        
    def forward(self, x, y): # 순전파 함수
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout): # 역전파 함수
        dx = dout * self.y # 서로의 값을 바꾼 것에 상류에서 구한 미분값 곱하기
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)
# backward, forward 이름에서 알 수 있듯이 호출 순서가 반대임을 유의하기

2.2 110.00000000000001 200


In [4]:
# 덧셈 노드 구현하기

class AddLayer:
    def __init___(self):
        pass
    
    def forward(self, x, y):# 덧셈 노드에서는 두 Input을 더해서 보냄
        out = x + y
        
        return out
    
    def backward(self, dout): # 상류에서 내려온 미분 값(dout)을 그대로 보냄
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 각각 필요한 계층 불러옴

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [6]:
# 활성화 함수와 역전파, 순전파에 대한 구현
class Relu:
    def __init__(self):
        self.mask = None
    
    # 순전파 부분 구현
    def forward(self, x):
        self.mask = (x <= 0) # 0 이하인 경우 True(= 1), 0 초과인 경우 False(= 0)
        out = x.copy()
        out[self.mask] = 0 # out[1] = 0, out[0] = 1
        
        return out
    
    def backward(self, dout): # 순전파에서 사용한 mask를 활용
        dout[self.mask] = 0 # 0 이하인 경우 미분값 = 0 -> dout[1] = 0 / 0 초과인 경우 False -> dout[0] = 이전 상류 미분 값을 그대로
        dx = dout
        
        return dx
    

In [7]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [8]:
mask = (x <= 0)
print(mask)
# 0 보다 작아야 True, 0 보다 커야 False

[[False  True]
 [ True False]]


In [9]:
# 시그모이드 함수

class Sigmoid: 
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

In [10]:
X = np.random.randn(2) # 0 혹은 1 
W = np.random.randn(2, 3)  # 가중치
B = np.random.randn(3) # 편향

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


In [11]:
# 신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서 어파인 변환이라고 함.
X_dot_W = np.array([[0,0,0], [10, 10, 10]])
B = np.array([1,2,3])

print(X_dot_W)

print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [12]:
dY = np.array([[1,2,3], [4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [13]:
# 어파인 변환 -> 오차역전파를 단순히 스칼라 값으로 수행하는 것이 아닌 행렬을 통해서 수행하고자 함.
# 이때,어파인의 의미는 행렬의 곱을 의미함.

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx

In [14]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [15]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x - c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

In [16]:
# 소프트맥스 함수 & 교차 엔트로피 오차(cross entropy error)을 통한 오차역전파 확인
# 소프트맥스 함수 & 교차 엔트로피 오차 // 항등 함수 & 오차제곱합(Sum Square Error)을 묶어서 사용 -> 오차 역전파가 차분으로 깔끔하게 계산
# 추론 과정에선 소프트맥스, 항등함수 안 씀 -> 우리가 관심있는 건 Affine의 output(= Score)가 가장 큰 것을 찾는 것이 중요함

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout = 1):
         batch_size = self.t.shape[0]
         dx = (self.y - self.t) / batch_size
         
         return dx

In [17]:
import numpy as np
from collections import OrderedDict

# Affine, Relu, and SoftmaxWithLoss classes should be defined or imported here

def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x + h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x)  # f(x - h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val  # 값 복원
        it.iternext()   
        
    return grad

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 초기화 함수로 각 인수는 순서대로 입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수, 가중치 초기화 시 정규분포의 스케일
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 1번째 층의 가중치
        self.params['b1'] = np.zeros(hidden_size)# 1번째 층의 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)# 2번째 층의 가중치
        self.params['b2'] = np.zeros(output_size)# 2번째 층의 편향
        
        # 계층 생성
        self.layers = OrderedDict() # 순서가 있는 딕셔너리 변수로, 신경망의 계층을 보관
        # 어파인 변환 - ReLu - 어파인 변환 순으로 계층 구조
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss() # 신경망의 마지막 계층으로 해당 실습에서는 SoftmaxWithLoss 활용함.
        
    def predict(self, x):
        # 예측(추론)을 수행함.
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def accuracy(self, x, t):
        # 정확도를 구한다.
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: 
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        # 가중치 매개변수의 기울기를 수치 미분 방식으로 구함, 이전 4장에서 한 것과 동일
        # x: 입력 데이터, t: 정답 레이블
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # 가중치 매개변수의 기울기를 오차역전파법으로 구함.
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        # 손실 함수의 값을 구함. x: 입력 데이터, t: 정답 레이블
        y = self.predict(x)
        return self.lastLayer.forward(y, t)


In [18]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
            return x
        
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = self.argmax(y, axis =  1)
        if t.ndim != 1 : t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout =1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [19]:
!pip install import_ipynb


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [20]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'dataset')))

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 전처리
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 모델 구축
model = Sequential([
    Flatten(input_shape=(28, 28)),  # 입력을 1차원으로 변환
    Dense(128, activation='relu'),  # 은닉층
    Dense(10, activation='softmax')  # 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 모델 훈련
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


In [24]:
# TensorFlow와 Keras를 사용한 MNIST 데이터셋 로드 및 모델 훈련
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 전처리
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 모델 구축
model = Sequential([
    Flatten(input_shape=(28, 28)),  # 입력을 1차원으로 변환
    Dense(128, activation='relu'),  # 은닉층
    Dense(10, activation='softmax')  # 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 모델 훈련
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

# TwoLayerNet 클래스와 관련된 전체 코드 포함
import numpy as np
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val
        it.iternext()
        
    return grad

# TensorFlow로 로드한 MNIST 데이터 재사용
t_train = y_train
t_test = y_test
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# TwoLayerNet 모델 초기화
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 배치 데이터 준비
x_batch = x_train[:3]
t_batch = t_train[:3]

# 수치 미분과 오차 역전파의 기울기 비교
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치 매개변수의 차이의 절댓값을 구하고 이를 평균한 값이 오차가 됨.
# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값의 평균을 계산
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":", diff)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step - accuracy: 0.8654 - loss: 0.4720 - val_accuracy: 0.9552 - val_loss: 0.1559
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 549us/step - accuracy: 0.9618 - loss: 0.1306 - val_accuracy: 0.9673 - val_loss: 0.1103
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 544us/step - accuracy: 0.9740 - loss: 0.0874 - val_accuracy: 0.9693 - val_loss: 0.1015
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 534us/step - accuracy: 0.9825 - loss: 0.0599 - val_accuracy: 0.9705 - val_loss: 0.0961
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 545us/step - accuracy: 0.9875 - loss: 0.0455 - val_accuracy: 0.9743 - val_loss: 0.0936
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 213us/step - accuracy: 0.9691 - loss: 0.1022
Test accuracy: 0.9732000231742859
W1: 3.8297385317136033e-10
b1: 2.2994719354205247e-09
W2: 6.227503958919009e-09
b2: 1.399209405020252e-07


In [25]:
# 오차역전파법을 사용한 학습 구현하기

iters_num = 10000 # 총 반복 횟수
train_size = x_train.shape[0] # 훈련 데이터의 크기
batch_size = 100 # 미니배치 크기
learning_rate = 0.1# 학습률 설정

train_loss_list = [] # 훈련 데이터의 손실 값을 저장하는 리스트
train_acc_list = [] # 각 에포크마다 훈련 정확도를 저장하는 리스트
test_acc_list = [] # 각 에포크마다 테스트 정확도를 저장하는 리스트

iter_per_epoch = max(train_size / batch_size, 1) # 한 에포크당 반복 횟수(데이터셋을 한 번 모두 사용하는 횟수)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size) # 무작위로 훈련 데이터에서 설정한 배치 크기만큼 선정
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch) # 기울기 계산 -> 오차 역전파를 활용해서 손실 함수의 기울기를 구하는 코드
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # 오차역전파로 구한 기울기에 학습률을 곱하여 최적의 가중치 매개변수로 조정
        
    loss = network.loss(x_batch, t_batch) # 현재 미니 배치 데이터, 정답 레이블에 대한 손실 함수 계산 -> 해당 실습에서는 교차 엔트로피 오차 사용
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)# 훈련 데이터에 대한 정확도 구하기
        test_acc = network.accuracy(x_test, t_test)# 시험 데이터에 대한 정확도 구하기
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
        
    #  훈련 데이터와 테스트 데이터 모두 높은 정확도를 보여주고 있음.

0.11163333333333333 0.1106
0.9036666666666666 0.9096
0.9247 0.9271
0.9377166666666666 0.9387
0.9469666666666666 0.944
0.9515333333333333 0.9484
0.9571166666666666 0.9555
0.9600833333333333 0.9575
0.9641333333333333 0.9599
0.9674666666666667 0.9642
0.9697833333333333 0.9646
0.9716833333333333 0.9665
0.9739666666666666 0.9679
0.97445 0.9693
0.9764166666666667 0.9697
0.9772666666666666 0.9707
0.9782833333333333 0.9697
